In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# Read Data

In [2]:
df1 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
df2 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')
df3 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv')
df4 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/supplemental_files/secondary_stock_prices.csv')
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(2332531, 12)
(269881, 12)
(2384575, 12)
(284583, 12)


In [3]:
df = pd.concat([df1,df2, df3, df4], axis = 0, ignore_index = True)
print(df.shape)
df

(5271570, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
5271565,20220624_9980,2022-06-24,9980,111.0,111.0,109.0,110.0,88000,1.0,NaN,False,-0.009009
5271566,20220624_9986,2022-06-24,9986,1862.0,1913.0,1862.0,1913.0,3500,1.0,NaN,False,-0.007495
5271567,20220624_9995,2022-06-24,9995,402.0,404.0,400.0,402.0,16000,1.0,NaN,False,0.004950
5271568,20220624_9996,2022-06-24,9996,1281.0,1281.0,1231.0,1241.0,1700,1.0,NaN,False,0.000806


In [4]:
# To get the column "AverageNumberOfShares" for each stock according to the latest "DisclosedDate"
df_fin1 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')
print(df_fin1.shape)
df_fin2 = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/supplemental_files/financials.csv')
print(df_fin2.shape)
df_fin = pd.concat([df_fin1, df_fin2], axis = 0, ignore_index = True)
print(df_fin.shape)
df_fin

C:\Users\jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(92956, 45)
(9794, 45)
(102750, 45)


,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0
2,2.016123e+13,20170104_4575,2017-01-04,4575.0,2017-01-04,12:00:00,1.483499e+09,ForecastRevision,2016-12-31,2Q,...,-93.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.017010e+13,20170105_2659,2017-01-05,2659.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,285.05,NaN,False,True,False,False,False,31981654.0,18257,31963405.0
4,2.017011e+13,20170105_3050,2017-01-05,3050.0,2017-01-05,15:30:00,1.483598e+09,ForecastRevision,2017-02-28,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102745,2.022062e+13,20220624_7438,2022-06-24,7438.0,2022-06-24,15:00:00,1.656050e+09,NumericalCorrection,2022-03-31,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102746,2.022062e+13,20220624_7678,2022-06-24,7678.0,2022-06-24,16:10:00,1.656055e+09,NumericalCorrection,2022-03-31,FY,...,NaN,NaN,True,True,False,True,False,5377070.0,50549,5325854.0
102747,2.022062e+13,20220624_8705,2022-06-24,8705.0,2022-06-24,15:00:00,1.656050e+09,NumericalCorrection,2022-03-31,FY,...,NaN,NaN,True,True,True,True,False,58251592.0,7868,58141036.0
102748,2.022062e+13,20220624_9522,2022-06-24,9522.0,2022-06-24,15:15:00,1.656051e+09,ForecastRevision,2022-12-31,FY,...,-57.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# To get the column "IssuedShares" for each stock if no information can be found from financials
df_num_share = pd.read_csv('./jpx-tokyo-stock-exchange-prediction/stock_list.csv')
df_num_share.head().T

,0,1,2,3,4
SecuritiesCode,1301,1305,1306,1308,1309
EffectiveDate,20211230,20211230,20211230,20211230,20211230
Name,"KYOKUYO CO.,LTD.",Daiwa ETF-TOPIX,NEXT FUNDS TOPIX Exchange Traded Fund,Nikko Exchange Traded Index Fund TOPIX,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...
Section/Products,First Section (Domestic),ETFs/ ETNs,ETFs/ ETNs,ETFs/ ETNs,ETFs/ ETNs
NewMarketSegment,Prime Market,NaN,NaN,NaN,NaN
33SectorCode,50,-,-,-,-
33SectorName,"Fishery, Agriculture and Forestry",-,-,-,-
17SectorCode,1,-,-,-,-
17SectorName,FOODS,-,-,-,-
NewIndexSeriesSizeCode,7,-,-,-,-


# Create New Columns to help calculation

##### Adjusted Volume

In [6]:
# Date with format "YYYY-MM"
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')
df.sort_values(by = ['Date', 'SecuritiesCode'], inplace=True, ignore_index=True)
df['Date_Month'] = df['Date'].dt.strftime("%Y-%m")

# To Adjust the Historical Volume before the split/reverse-split
# Adjusted_Vol = Volume / AdjustmentFactor
# It is possible that a stock encountered multiple split/reverse-split.
# So the cummulative effect of adjustment factor should be considered
df['Cum_Adj_Factor'] = df.groupby('SecuritiesCode')['AdjustmentFactor'].transform(lambda x: np.cumprod(x[::-1])[::-1])
df['Adj_Vol'] = df['Volume'] / df['Cum_Adj_Factor']

#### Adjusted Daily Return

In [7]:
# The "Target" column from "stock_price.csv" means Change ratio of adjusted closing price 
# between t+2 and t+1 where t+0 is TradeDate.
# The daily return can be obtained by shifting "Target" column downwards by 2 rows.
df['Daily_Return'] = df.groupby('SecuritiesCode')['Target'].shift(2)
df['Log_Return'] = np.log(df['Daily_Return'] + 1)

###### Example Checking

In [8]:
# Example Checking
df.query("SecuritiesCode == 3962 and Date > '2020-08-25' and Date < '2020-12-31'")\
[['Date', 'Close', 'Volume', 'AdjustmentFactor', 'Cum_Adj_Factor', 'Adj_Vol', 'Target', 'Daily_Return', 'Log_Return']]

,Date,Close,Volume,AdjustmentFactor,Cum_Adj_Factor,Adj_Vol,Target,Daily_Return,Log_Return
3452560,2020-08-26,14100.0,551000,1.0,0.25,2204000.0,-0.035311,0.028446,0.028049
3456547,2020-08-27,14160.0,721400,0.5,0.25,2885600.0,0.086384,0.004255,0.004246
3460533,2020-08-28,6830.0,1566600,1.0,0.50,3133200.0,0.099730,-0.035311,-0.035949
3464519,2020-08-31,7420.0,1251900,1.0,0.50,2503800.0,0.040441,0.086384,0.082854
3468504,2020-09-01,8160.0,2016700,1.0,0.50,4033400.0,-0.017668,0.099730,0.095065
...,...,...,...,...,...,...,...,...,...
3780657,2020-12-24,7880.0,494800,1.0,0.50,989600.0,-0.042581,0.010256,0.010204
3784687,2020-12-25,7750.0,370400,1.0,0.50,740800.0,-0.024259,-0.016497,-0.016635
3788719,2020-12-28,7420.0,638400,0.5,0.50,1276800.0,-0.026243,-0.042581,-0.043514
3792751,2020-12-29,3620.0,1015100,1.0,1.00,1015100.0,0.062411,-0.024259,-0.024558


#### Number of Shares Issued

In [9]:
# Find the latest available ShareIssued information and then adjusted by Cum_Adj_Factor as above
# We only care about the ShareIssued on the last trading day in each month for calculating Market_Cap. 

# ShareIssued Information from financials
df_fin.rename(columns = {'AverageNumberOfShares' : 'IssuedShares'}, inplace = True)
df_fin.dropna(subset = ['SecuritiesCode'], inplace = True)
df_fin.dropna(subset = ['IssuedShares'], inplace = True)
df_fin = df_fin[pd.to_numeric(df_fin['IssuedShares'], errors='coerce').notnull()]
df_fin['Date'] = pd.to_datetime(df_fin['DisclosedDate'], format = '%Y-%m-%d')
df_fin['IssuedShares'] = df_fin['IssuedShares'].astype('float')
df_fin['SecuritiesCode'] = df_fin['SecuritiesCode'].astype('int')


# ShareIssued Information from stock_list.csv
df_num_share.rename(columns = {'TradeDate' : 'Date'}, inplace = True )
df_num_share['Date'] = pd.to_datetime(df_num_share['Date'], format = '%Y%m%d')

# Combine information from both tables
df_num_share = pd.concat([df_fin[['Date', 'SecuritiesCode', 'IssuedShares']],\
                          df_num_share[['Date', 'SecuritiesCode', 'IssuedShares']]], \
                          axis = 0, ignore_index=True)
df_num_share.drop_duplicates(inplace = True)

# Fill the IssuedShares for each stock on each trading day
df = pd.merge(df, df_num_share, on = ['Date', 'SecuritiesCode' ], how = 'left')
df['IssuedShares'] = df['IssuedShares'] / df['Cum_Adj_Factor']
df['IssuedShares'] = df.groupby('SecuritiesCode')['IssuedShares'].ffill()

# Fill the remaining empty IssuedShares by mean 
df['IssuedShares'] = df.groupby('SecuritiesCode')['IssuedShares'].transform(lambda x: x.fillna(x.mean()))


C:\Users\jack\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\jack\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\jack\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [10]:
df.query('SecuritiesCode == 3962')[['Date_Month', 'IssuedShares']].value_counts(sort = False).head(20)

Date_Month  IssuedShares
2017-01     5.933264e+07    19
2017-02     5.100342e+07    11
            5.933264e+07     9
2017-03     5.100342e+07    22
2017-04     5.100342e+07    20
2017-05     5.100342e+07     6
            5.112786e+07    14
2017-06     5.112786e+07    22
2017-07     5.112786e+07    20
2017-08     5.112786e+07     8
            5.131203e+07    14
2017-09     5.131203e+07    20
2017-10     5.131203e+07    21
2017-11     5.131203e+07     8
            5.144552e+07    12
2017-12     5.144552e+07    21
2018-01     5.144552e+07    19
2018-02     5.144552e+07     8
            5.311976e+07    11
2018-03     5.311976e+07    21
dtype: int64

# Calculate ATV

#### Aggregate the Data to Monthly Data

In [11]:
df_monthly = df.groupby(['SecuritiesCode', 'Date_Month'], as_index = False)[['Adj_Vol', 'Log_Return']].sum()
df_monthly['Total_Market_Vol'] = df_monthly.groupby('Date_Month')['Adj_Vol'].transform('sum')
df_monthly['Constant'] = 1
df_monthly.rename( columns = {'Log_Return' : 'Monthly_Return'}, inplace = True)
df_monthly

,SecuritiesCode,Date_Month,Adj_Vol,Monthly_Return,Total_Market_Vol,Constant
0,1301,2017-01,357500.0,-0.005861,2.976180e+10,1
1,1301,2017-02,607000.0,0.087532,3.118322e+10,1
2,1301,2017-03,1917900.0,-0.011850,3.216194e+10,1
3,1301,2017-04,660900.0,-0.000341,3.127204e+10,1
4,1301,2017-05,775900.0,0.040072,3.209785e+10,1
...,...,...,...,...,...,...
260734,25935,2022-02,217800.0,0.026125,3.210268e+10,1
260735,25935,2022-03,438100.0,0.027469,4.167630e+10,1
260736,25935,2022-04,772100.0,-0.043585,3.281563e+10,1
260737,25935,2022-05,201300.0,-0.024942,3.537264e+10,1


In [12]:
df_monthly[['Date_Month','Total_Market_Vol']].value_counts(sort = False)

Date_Month  Total_Market_Vol
2017-01     2.976180e+10        3758
2017-02     3.118322e+10        3760
2017-03     3.216194e+10        3786
2017-04     3.127204e+10        3787
2017-05     3.209785e+10        3785
                                ... 
2022-02     3.210268e+10        4129
2022-03     4.167630e+10        4119
2022-04     3.281563e+10        4104
2022-05     3.537264e+10        4098
2022-06     3.102794e+10        4092
Length: 66, dtype: int64

#### Fit Linear Model 

In [13]:
def helper(x, window_length = 12):
    # Given volume data of a stock, fit a linear regression model by using past 12 month data
    # to obtain coefficients for each month
    # i.e: For coefficients on month 12, the data between month 1 and month 12 are used.
    if len(x) >= window_length:
        res = RollingOLS(x['Adj_Vol'], x[['Total_Market_Vol', 'Constant']],  window = window_length).fit().params
    # If do not have enough data, cannot estimate the coefficients. Return empty dataframe
    else:
        res = pd.DataFrame({'Total_Market_Vol':[np.nan]*len(x),
                            'Constant':[np.nan]*len(x)})
    return res

dfs = [helper(df_sub) for stock, df_sub in df_monthly.groupby(['SecuritiesCode'])]
dfs = pd.concat(dfs, axis = 0, ignore_index=True)
dfs.rename(columns = {'Total_Market_Vol' : 'Coef_B',
                      'Constant' : 'Coef_A'}, inplace=True)
df_monthly = pd.concat([df_monthly, dfs], axis = 1)

C:\Users\jack\Anaconda3\lib\site-packages\statsmodels\regression\rolling.py:265: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood


In [14]:
df_monthly.tail(60)

,SecuritiesCode,Date_Month,Adj_Vol,Monthly_Return,Total_Market_Vol,Constant,Coef_B,Coef_A
260679,25935,2017-07,407900.0,-0.026243,3.196177e+10,1,NaN,NaN
260680,25935,2017-08,278600.0,-0.008105,3.218699e+10,1,NaN,NaN
260681,25935,2017-09,512700.0,-0.040543,3.114301e+10,1,NaN,NaN
260682,25935,2017-10,523600.0,0.011893,3.710545e+10,1,NaN,NaN
260683,25935,2017-11,495500.0,0.017579,3.914558e+10,1,NaN,NaN
260684,25935,2017-12,462600.0,0.037059,3.528188e+10,1,-1.019225e-05,9.285085e+05
260685,25935,2018-01,474900.0,0.052249,3.479029e+10,1,-1.930690e-05,1.240438e+06
260686,25935,2018-02,581300.0,0.037363,3.620878e+10,1,-2.515307e-05,1.458433e+06
260687,25935,2018-03,640700.0,-0.011581,3.337377e+10,1,-1.029046e-05,9.063961e+05
260688,25935,2018-04,1051800.0,-0.003890,3.134722e+10,1,-1.459845e-05,1.063763e+06


####  ATV

In [15]:
df_monthly['ATV'] = df_monthly['Adj_Vol'] - df_monthly['Coef_A'] - df_monthly['Coef_B']*df_monthly['Total_Market_Vol']

# Calculate Market Cap

In [16]:
# To ensure the last trading day of each month contains the latest available close price
df['Close'] = df.groupby('SecuritiesCode')['Close'].ffill()
df_t = df.groupby(['SecuritiesCode', 'Date_Month'], as_index = False).tail(1)
df_t.rename(columns = {'Close' : 'Latest_Close_in_Current_Month'}, inplace = True)
df_monthly = df_monthly.merge(df_t[['SecuritiesCode', 'Date_Month', \
                                    'IssuedShares', 'Latest_Close_in_Current_Month']],\
                             on = ['SecuritiesCode', 'Date_Month'], how = 'left')
df_monthly['Market_Cap'] = df_monthly['IssuedShares'] * df_monthly['Latest_Close_in_Current_Month']

C:\Users\jack\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
df_monthly

,SecuritiesCode,Date_Month,Adj_Vol,Monthly_Return,Total_Market_Vol,Constant,Coef_B,Coef_A,ATV,IssuedShares,Latest_Close_in_Current_Month,Market_Cap
0,1301,2017-01,357500.0,-0.005861,2.976180e+10,1,NaN,NaN,NaN,1.070933e+07,2722.0,2.915081e+10
1,1301,2017-02,607000.0,0.087532,3.118322e+10,1,NaN,NaN,NaN,1.050302e+07,2971.0,3.120448e+10
2,1301,2017-03,1917900.0,-0.011850,3.216194e+10,1,NaN,NaN,NaN,1.050302e+07,2936.0,3.083687e+10
3,1301,2017-04,660900.0,-0.000341,3.127204e+10,1,NaN,NaN,NaN,1.050302e+07,2935.0,3.082637e+10
4,1301,2017-05,775900.0,0.040072,3.209785e+10,1,NaN,NaN,NaN,1.050296e+07,3055.0,3.208654e+10
...,...,...,...,...,...,...,...,...,...,...,...,...
260734,25935,2022-02,217800.0,0.026125,3.210268e+10,1,0.000028,-608780.245421,-86176.892189,3.424696e+07,1939.0,6.640486e+10
260735,25935,2022-03,438100.0,0.027469,4.167630e+10,1,0.000021,-349596.304243,-70031.521606,3.424696e+07,1993.0,6.825420e+10
260736,25935,2022-04,772100.0,-0.043585,3.281563e+10,1,0.000017,-207944.634666,423064.391324,3.424696e+07,1908.0,6.534320e+10
260737,25935,2022-05,201300.0,-0.024942,3.537264e+10,1,0.000020,-337376.089985,-163215.846520,3.424696e+07,1861.0,6.373360e+10


# Classify the Stocks into 4 Portfolios

#### Obtain Previous ATV and Market Cap for classification

In [18]:
df_monthly['Prev_ATV'] = df_monthly.groupby('SecuritiesCode')['ATV'].shift()
df_monthly['Prev_Market_Cap'] = df_monthly.groupby('SecuritiesCode')['Market_Cap'].shift()

#### Calculate Median of ATV and Market Cap for Previous Month

In [19]:
df_monthly['Median_Prev_ATV'] = df_monthly.groupby('Date_Month')['Prev_ATV'].transform('median')
df_monthly['Median_Prev_Market_Cap'] = df_monthly.groupby('Date_Month')['Prev_Market_Cap'].transform('median')

#### Classification

In [20]:
cond_list = [(df_monthly['Prev_Market_Cap'] < df_monthly['Median_Prev_Market_Cap'])&(df_monthly['Prev_ATV'] >= df_monthly['Median_Prev_ATV']),  #SO
             (df_monthly['Prev_Market_Cap'] < df_monthly['Median_Prev_Market_Cap'])&(df_monthly['Prev_ATV'] < df_monthly['Median_Prev_ATV']),   #SL
             (df_monthly['Prev_Market_Cap'] >= df_monthly['Median_Prev_Market_Cap'])&(df_monthly['Prev_ATV'] >= df_monthly['Median_Prev_ATV']),   #BO
             (df_monthly['Prev_Market_Cap'] >= df_monthly['Median_Prev_Market_Cap'])&(df_monthly['Prev_ATV'] < df_monthly['Median_Prev_ATV'])]    #BL

df_monthly['Portfolio_Type'] = np.select(cond_list, ['SO', 'SL', 'BO', 'BL'], 'NA')

In [21]:
# Checking
df_monthly.query("Date_Month >= '2018-01'").groupby('Date_Month')['Portfolio_Type'].value_counts().head(20)

Date_Month  Portfolio_Type
2018-01     BL                1136
            SO                1130
            BO                 728
            SL                 723
            NA                 131
2018-02     BL                1127
            SO                1124
            BO                 734
            SL                 733
            NA                 130
2018-03     BO                1059
            SL                1055
            BL                 818
            SO                 815
            NA                 116
2018-04     BO                 985
            SL                 980
            BL                 892
            SO                 886
            NA                 121
Name: Portfolio_Type, dtype: int64

# Calculate OML

#### Calculate Portfolio Weight for Each Stock

In [22]:
df_monthly['Portfolio_Market_Cap'] = df_monthly.groupby(['Date_Month', 'Portfolio_Type'])\
                                     ['Prev_Market_Cap'].transform('sum')
df_monthly['Portfolio_Weight'] = df_monthly['Prev_Market_Cap'] / df_monthly['Portfolio_Market_Cap']

#### Calculate Weighted Retrun for each portfolio in each month and OML

In [23]:
df_monthly['Weighted_Return'] = df_monthly['Monthly_Return'] * df_monthly['Portfolio_Weight']
df_portfolio_return = df_monthly.groupby(['Date_Month', 'Portfolio_Type'], as_index = False)\
['Weighted_Return'].sum()
df_portfolio_return = df_portfolio_return.pivot_table(index = 'Date_Month', columns = 'Portfolio_Type',\
                                values = 'Weighted_Return')
df_portfolio_return['OML'] = ((df_portfolio_return['BO'] + df_portfolio_return['SO']) -\
                              (df_portfolio_return['BL'] + df_portfolio_return['SL']))/2

In [24]:
df_portfolio_return.tail(55)

Portfolio_Type,BL,BO,NA,SL,SO,OML
Date_Month,,,,,,
2017-12,NaN,NaN,0.012582,NaN,NaN,NaN
2018-01,0.015015,0.022046,0.056812,0.048739,0.042535,0.000414
2018-02,-0.032611,-0.037966,-0.040703,-0.030913,-0.022794,0.001382
2018-03,-0.027011,-0.029795,-0.015250,-0.031841,-0.035113,-0.003028
2018-04,0.036915,0.031570,-0.001615,0.000014,-0.011431,-0.008395
2018-05,-0.019457,-0.005732,0.007420,-0.015176,-0.006028,0.011436
2018-06,-0.007732,0.006966,-0.019057,-0.033770,-0.017623,0.015423
2018-07,-0.003623,0.020989,-0.011917,-0.002662,-0.010539,0.008368
2018-08,-0.023583,0.000518,0.004812,-0.013808,-0.005817,0.016046


In [25]:
df_portfolio_return['OML'].describe()

count    54.000000
mean      0.000634
std       0.014108
min      -0.030352
25%      -0.008180
50%       0.000533
75%       0.010350
max       0.050725
Name: OML, dtype: float64

# Calculate OML Ver2

#### Use Total_Market_Cap instead of Portfolio_Market_Cap to calculate the weight

In [26]:
df_monthly['Total_Prev_Market_Cap'] = df_monthly.groupby(['Date_Month'])\
                                     ['Prev_Market_Cap'].transform('sum')
df_monthly['Portfolio_Weight2'] = df_monthly['Prev_Market_Cap'] / df_monthly['Total_Prev_Market_Cap']

#### Calculate Weighted Retrun for each portfolio in each month and OML

In [27]:
df_monthly['Weighted_Return2'] = df_monthly['Monthly_Return'] * df_monthly['Portfolio_Weight2']
df_portfolio_return2 = df_monthly.groupby(['Date_Month', 'Portfolio_Type'], as_index = False)\
['Weighted_Return2'].sum()
df_portfolio_return2 = df_portfolio_return2.pivot_table(index = 'Date_Month', columns = 'Portfolio_Type',\
                                values = 'Weighted_Return2')
df_portfolio_return2['OML'] = ((df_portfolio_return2['BO'] + df_portfolio_return2['SO']) -\
                              (df_portfolio_return2['BL'] + df_portfolio_return2['SL']))/2

In [28]:
df_portfolio_return2.tail(55)

Portfolio_Type,BL,BO,NA,SL,SO,OML
Date_Month,,,,,,
2017-12,NaN,NaN,0.012582,NaN,NaN,NaN
2018-01,0.011029,0.005319,0.000404,3.432104e-04,0.000428,-0.002813
2018-02,-0.021170,-0.012377,-0.000293,-2.291354e-04,-0.000233,0.004394
2018-03,-0.008057,-0.020212,-0.000085,-2.959534e-04,-0.000297,-0.006078
2018-04,0.014925,0.018042,-0.000011,1.332519e-07,-0.000095,0.001511
2018-05,-0.012732,-0.001844,0.000053,-1.147005e-04,-0.000056,0.005473
2018-06,-0.004083,0.003120,-0.000135,-2.810250e-04,-0.000152,0.003666
2018-07,-0.001700,0.010626,-0.000094,-2.295297e-05,-0.000085,0.006132
2018-08,-0.011560,0.000251,0.000042,-1.194023e-04,-0.000044,0.005943


In [29]:
df_portfolio_return2['OML'].describe()

count    54.000000
mean      0.000206
std       0.007977
min      -0.027068
25%      -0.003751
50%      -0.000076
75%       0.004525
max       0.021954
Name: OML, dtype: float64

# Comparison

In [30]:
df_portfolio_return2.rename(columns = {'OML' : 'OML2'}, inplace = True)
df_compare = pd.merge(df_portfolio_return['OML'], df_portfolio_return2['OML2'],\
                      left_index = True, right_index = True, how = 'left')

In [31]:
df_compare.tail(55)

,OML,OML2
Date_Month,,
2017-12,NaN,NaN
2018-01,0.000414,-0.002813
2018-02,0.001382,0.004394
2018-03,-0.003028,-0.006078
2018-04,-0.008395,0.001511
2018-05,0.011436,0.005473
2018-06,0.015423,0.003666
2018-07,0.008368,0.006132
2018-08,0.016046,0.005943


In [32]:
df_compare.describe()

,OML,OML2
count,54.000000,54.000000
mean,0.000634,0.000206
std,0.014108,0.007977
min,-0.030352,-0.027068
25%,-0.008180,-0.003751
50%,0.000533,-0.000076
75%,0.010350,0.004525
max,0.050725,0.021954
